# 6. Analysis of the Calculations and the Input File 🔎
<br>

When running the command, it shows you the series of steps it did to create our input file, it is mainly divided into two sections:

<div style="line-height: 1.5;">

## <span style="color:SteelBlue;"> Gas Calculation</span>
- It considers our solute in the gas phase. This step is **necessary to calculate the solvation energy**.
<dir>

<div style="line-height: 1.5;">

## <span style="color:#2E8B57;"> CPCM Calculation</span>
- CPCM stands for **Conductor-like Polarizable Continuum Model**. It is a popular computer-based chemistry model that shows how a solvent affects a solute molecule.<br> 
- Before, we were considering our molecule in the gas phase, now we introduce a solvent that is treated as a continuous polarizable medium surrounding the solute molecule. The boundary between the solute and solvent is defined by a cavity, which is typically shaped to fit the solute molecule.
- It is an iterative method where the program uses several models to sort the conformers by energy and filters out the ones that have the lowest energy. In the end, we end up with the conformer that is most likely to exist in nature, i,e. the most stable.
- These calculations are **necessary to calculate activity coeffcients** 

<dir>

<div style="line-height: 1.5;">

## Steps taken
Here is a brief explanation of what each step means:

- **<span style="color:SteelBlue">number_of_conformers_generated: 50</span>** the number of conformers is the number of ways that our molecule can be arranged in space, this specifies the number of conformers that the algorithm attempts to generate for the molecule. Making different versions of a molecule let us see what the best and most stable ones might be. The number of conformers is specified by the number of rotable bonds in our molecule and the number is specified by the heuristic referenced [here](https://doi.org/10.1021/ci2004658). Here is a simplified overview of the criteria:

    - If there are 7 or fewer rotatable bonds, the code generates 50 conformers
    - If there are between 8 and 12 rotatable bonds, the code generates 200 conformers.
    - If there are 13 or more rotatable bonds, the code generates 300 conformers.

Rotatable bonds are single covalent bonds between atoms in a molecule. These bonds allow for free rotation around the bond axis. This makes the molecule more flexible. It allows for different spatial arrangements or conformations. <br><br>


- **<span style="color:SteelBlue">rms_threshold: 1.0</span>** this sets the RMS (root mean square) deviation threshold for filtering conformers. This threshold is used to work out how similar conformers can be before they are marked as duplicates. A lower RMS threshold means that only conformers with very similar structures will be filtered out, while a higher threshold allows for more variation.<br><br>

- **<span style="color:SteelBlue">rms_only_heavy_atoms</span>** the RMS calculation should only consider heavy atoms (non-hydrogen atoms). It is more useful to focus on heavy atoms when comparing structures, as hydrogen positions can vary more and are less important for the overall shape of the molecule

- **<span style="color:SteelBlue">rdkit_generate_conformers</span>** after the molecule's structure has been set, the program generates multiple conformers for the molecule using RDKit's distance geometry methods. Since the number of conformers is set to 50 for this molecule, 50 conformers will be generated.
    
- **<span style="color:SteelBlue">sort_energy</span>** sorts the generated conformers by their calculated energy, making sure that the conformers with the lowest energy, i.e. the most stable ones, are prioritized.

- **<span style="color:SteelBlue">filter_by_energy_window</span>** filters conformers to retain only those within a specified energy window.
    - The energy window is used to filter out conformers that are too high in energy. The  window that we are considering is: 6 * kJ_per_kcal.

- **<span style="color:SteelBlue">filter_by_rms_window</span>** filters conformers based on RMSD or Root Mean Square Deviation, which is a measure to assess the similarity between two molecular structures by calculating the average distance between corresponding atoms. Here we remove the conformers that are too similar to others, based on the rms threshold we set up (1.0 Å). This threshold is used for the positions of heavy atoms (non-hydrogen atoms).

- **<span style="color:SteelBlue;">ORCA_DFT_FAST</span>** performs a quick **Density Functional Theory** or DFT calculation using ORCA, to estimate the energy of conformers. <br>
    - **What is DFT** ❓<br>
        - DFT is used to calculate the electronic structure of atoms, helping understand how they behave and what properties they have. It calculates the total energy of a system based on how many electrons it has. This energy can be used to predict the structure of molecules, how much energy a reaction needs, and other properties.

        ![e](edensity.png)
        
- **<span style="color:SteelBlue">sort_energy</span>** organizes the conformers based on their calculated energies. The idea is to focus on conformers with lower energies, as these are usually more stable.

- **<span style="color:SteelBlue">filter</span>** filters the number of conformers to a specified maximum.

- **<span style="color:SteelBlue">ORCA_DFT_final</span>** performs a more thorough DFT calculation with ORCA, using a more accurate DFT method to refine energy calculations:

    - The "more thorough" calculation involves using a detailed DFT method with high-quality basis sets to improve the energy calculations of the different forms. This makes sure that the final energy values are as accurate as possible. It is similar to ORCA_DFT_FAST, but it has extra steps for single-point calculations after geometry optimization.

    - The **Single-Point Energy Calculations** calculates the energy of the optimized geometry without any more geometry changes, providing a precise energy value for the most stable conformer.
        
    - Uses larger and more accurate basis sets to improve the precision of the calculations, for example the [def2-TZVP](https://sites.google.com/site/orcainputlibrary/basis-sets) basis set. Briefly, the prefix def2 indicates that this basis set is part of the second generation of the Karlsruhe basis sets developed by the research group of Frank Weigend and Reinhart Ahlrichs. The TZV part stands for "Triple-Zeta Valence", indicating that the basis set contains three functions for each valence orbital, this sets are used in quantum chemistry to provide a more accurate description of electron distributions. The P indicates that polarization functions are included. These functions allow the electron cloud to distort more easily. This is important for accurately modelling molecular geometries and interactions.

- **<span style="color:#2E8B57">orca_xtb2_alpb</span>** step used exclusively in CPCM calculations. It performs an XTB2 ALPB calculation with ORCA using a semi-empirical method to estimate solvation effects <br>
    - **Extended Tight Binding (XTB)** is a semi-empirical quantum chemistry method. It has been developed to provide an optimal balance between computational efficiency and accuracy for large molecular systems. **XTB2** refers to the second version of the XTB method, incorporating improvements over the original XTB method to enhance accuracy.<br><br>
    - **ALPB** it is a solvation model that contributes in determining which conformer has the lowest energy, to later help us decide which conformer to choose. It treats the solvent as a continuous medium interacting with the solute <br><br>

- **<span style="color:#2E8B57">ORCA_DFT_CPCM_fast</span>** step used exclusively in CPCM calculations. It is designed to  estimate the energies of conformers while accounting for effects caused by the presence of a solvent using the CPCM method in combination with DFT. Combining DFT with CPCM means that the calculation considers both the electronic structure and the effects of solvation. This provides a more realistic estimate of the energy for molecules in solution. <br><br>

- **<span style="color:#2E8B57">ORCA_DFT_CPCM_final</span>** step used exclusively in CPCM calculations. Tt performs the same calculations as described before, providing final energy values for the most stable conformers, including solvation effects.
<dir>